In [1]:
import geopandas as gpd

# Load the GeoJSON file
geojson_path = 'train_data/challenge_5_bb_TRAIN.geojson'
gdf = gpd.read_file(geojson_path)
gdf

,geometry
0,"POLYGON ((-90.23406 29.05768, -89.28282 29.242..."


In [2]:
import pyproj

def get_utm_zone(longitude):
    return int((longitude + 180) / 6) + 1

# Get the bounds of the geometry
minx, miny, maxx, maxy = gdf.geometry.bounds.iloc[0]

# Calculate UTM zone
utm_zone = get_utm_zone(minx)

# Check for a suitable projection using pyproj
proj = pyproj.Proj(proj='utm', zone=utm_zone, ellps='WGS84')

# Get the corresponding EPSG code for the UTM zone using pyproj
utm_crs = pyproj.CRS(f"+proj=utm +zone={utm_zone} +datum=WGS84")
epsg_code = utm_crs.to_epsg()

# Reproject the GeoDataFrame to the chosen EPSG code
gdf = gdf.to_crs(epsg=epsg_code)
gdf

,geometry
0,"POLYGON ((769308.177 3217535.091, 861338.486 3..."


In [3]:
# import numpy as np

# # Create a grid of points 5120m apart
# x = np.arange(gdf.total_bounds[0], gdf.total_bounds[2], 2560)
# y = np.arange(gdf.total_bounds[1], gdf.total_bounds[3], 2560)
# xx, yy = np.meshgrid(x, y)
# points = np.vstack([xx.ravel(), yy.ravel()]).T

# grid = gpd.GeoDataFrame(geometry=gpd.points_from_xy(points[:, 0], points[:, 1], crs=gdf.crs))
# grid

In [4]:
# import pandas as pd
# import geopandas as gpd
# from shapely.geometry import Point
# import pystac_client
# import stackstac
# import torch
# from torchvision import transforms as v2
# from box import Box
# import yaml
# import math
# from rasterio.enums import Resampling
# from tqdm import tqdm
# import rasterio
# import warnings
# import os
# import numpy as np
# import rioxarray  # Make sure to import rioxarray to extend xarray

# from src.model import ClayMAEModule

# warnings.filterwarnings("ignore")

# STAC_API = "https://earth-search.aws.element84.com/v1"
# COLLECTION = "sentinel-2-l2a"

# # Load the model and metadata
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# ckpt = "https://clay-model-ckpt.s3.amazonaws.com/v0.5.7/mae_v0.5.7_epoch-13_val-loss-0.3098.ckpt"
# torch.set_default_device(device)

# torch.cuda.empty_cache()  # Clear GPU cache

# # Assuming grid is a GeoDataFrame with the points
# points = grid.to_crs("EPSG:4326").geometry.apply(lambda x: (x.x, x.y)).tolist()

# model = ClayMAEModule.load_from_checkpoint(
#     ckpt, metadata_path="configs/metadata.yaml", shuffle=False, mask_ratio=0
# )
# model.eval()
# model = model.to(device)

# metadata = Box(yaml.safe_load(open("configs/metadata.yaml")))

# # Function to normalize timestamp
# def normalize_timestamp(date):
#     week = date.isocalendar().week * 2 * np.pi / 52
#     hour = date.hour * 2 * np.pi / 24
#     return (math.sin(week), math.cos(week)), (math.sin(hour), math.cos(hour))

# # Function to normalize lat/lon
# def normalize_latlon(lat, lon):
#     lat = lat * np.pi / 180
#     lon = lon * np.pi / 180
#     return (math.sin(lat), math.cos(lat)), (math.sin(lon), math.cos(lon))

# def to_device(data, device):
#     if isinstance(data, torch.Tensor):
#         return data.to(device)
#     elif isinstance(data, dict):
#         return {k: to_device(v, device) for k, v in data.items()}
#     elif isinstance(data, list):
#         return [to_device(v, device) for v in data]
#     return data

# def process_point(lon, lat, model, metadata, year, device, j):
#     model.to(device)  # Ensure the model is on the correct device
#     catalog = pystac_client.Client.open(STAC_API)
#     search = catalog.search(
#         collections=[COLLECTION],
#         datetime=f"{year}-08-01/{year}-09-30",
#         bbox=(lon - 1e-5, lat - 1e-5, lon + 1e-5, lat + 1e-5),
#         max_items=10,
#         query={"eo:cloud_cover": {"lt": 80}},
#     )

#     all_items = search.get_all_items()
#     items = list(all_items)
#     if not items:
#         return None
    
#     items = sorted(items, key=lambda x: x.properties.get('eo:cloud_cover', float('inf')))
#     lowest_cloud_item = items[0]

#     epsg = lowest_cloud_item.properties["proj:epsg"]

#     poidf = gpd.GeoDataFrame(
#         pd.DataFrame(),
#         crs="EPSG:4326",
#         geometry=[Point(lon, lat)],
#     ).to_crs(epsg)

#     coords = poidf.iloc[0].geometry.coords[0]

#     size = 256
#     gsd = 10
#     bounds = (
#         coords[0] - (size * gsd) // 2,
#         coords[1] - (size * gsd) // 2,
#         coords[0] + (size * gsd) // 2,
#         coords[1] + (size * gsd) // 2,
#     )

#     stack = stackstac.stack(
#         lowest_cloud_item,
#         bounds=bounds,
#         snap_bounds=False,
#         epsg=epsg,
#         resolution=gsd,
#         dtype="float32",
#         rescale=False,
#         fill_value=0,
#         assets=["blue", "green", "red", "nir"],
#         resampling=Resampling.nearest,
#     )

#     stack = stack.compute()

#     items = []
#     dates = []
#     for item in all_items:
#         if item.datetime.date() not in dates:
#             items.append(item)
#             dates.append(item.datetime.date())

#     platform = "sentinel-2-l2a"
#     mean = []
#     std = []
#     waves = []
#     for band in stack.band:
#         mean.append(metadata[platform].bands.mean[str(band.values)])
#         std.append(metadata[platform].bands.std[str(band.values)])
#         waves.append(metadata[platform].bands.wavelength[str(band.values)])

#     transform = v2.Compose([v2.Normalize(mean=mean, std=std)])

#     datetimes = stack.time.values.astype("datetime64[s]").tolist()
#     times = [normalize_timestamp(dat) for dat in datetimes]
#     week_norm = [dat[0] for dat in times]
#     hour_norm = [dat[1] for dat in times]

#     latlons = [normalize_latlon(lat, lon)] * len(times)
#     lat_norm = [dat[0] for dat in latlons]
#     lon_norm = [dat[1] for dat in latlons]

#     pixels = torch.from_numpy(stack.data.astype(np.float32)).to(device)
#     pixels = transform(pixels)

#     batch_size = 16
#     num_batches = math.ceil(len(stack) / batch_size)
    
#     embeddings_list = []
#     for i in range(num_batches):
#         start_idx = i * batch_size
#         end_idx = min((i + 1) * batch_size, len(stack))
        
#         batch_pixels = pixels[start_idx:end_idx].to(device)
#         batch_time = torch.tensor(np.hstack((week_norm, hour_norm))[start_idx:end_idx], dtype=torch.float32).to(device)
#         batch_latlon = torch.tensor(np.hstack((lat_norm, lon_norm))[start_idx:end_idx], dtype=torch.float32).to(device)
        
#         batch_datacube = {
#             "platform": platform,
#             "time": batch_time,
#             "latlon": batch_latlon,
#             "pixels": batch_pixels,
#             "gsd": torch.tensor(stack.gsd.values).to(device),
#             "waves": torch.tensor(waves).to(device),
#         }

#         batch_datacube = to_device(batch_datacube, device)

#         try:
#             model = model.to(device)

#             with torch.no_grad():
#                 unmsk_patch, _, _, _ = model.model.encoder(batch_datacube)
#             batch_embeddings = unmsk_patch[:, 0, :].cpu().numpy()
#             embeddings_list.append(batch_embeddings)
#         except RuntimeError as e:
#             if "out of memory" in str(e):
#                 print(f"GPU OOM for point ({lon}, {lat}), batch {i+1}/{num_batches}. Trying CPU...")
#                 device = torch.device("cpu")
#                 batch_datacube = to_device(batch_datacube, device)
#                 model = model.to(device)
#                 with torch.no_grad():
#                     unmsk_patch, _, _, _ = model.model.encoder(batch_datacube)
#                 batch_embeddings = unmsk_patch[:, 0, :].numpy()
#                 embeddings_list.append(batch_embeddings)
#                 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#             else:
#                 raise e

#     embeddings = np.concatenate(embeddings_list, axis=0)
#     return embeddings

# # Specify the years for the datetime range in the search
# years = [2018, 2019]

# # Initialize an empty dictionary to store results for both years
# results_dict = {"lon": [], "lat": [], f"embeddings_{years[0]}": [], f"embeddings_{years[1]}": []}

# # Iterate through the points and process each one for both years
# for i, point in enumerate(tqdm(points)):
#     lon, lat = point
#     results_dict["lon"].append(lon)
#     results_dict["lat"].append(lat)
    
#     for year in years:
#         embeddings = process_point(lon, lat, model, metadata, year, device, i)
#         if embeddings is not None:
#             results_dict[f"embeddings_{year}"].append(embeddings)
#         else:
#             results_dict[f"embeddings_{year}"].append(None)

# # Create a DataFrame from the results
# df = pd.DataFrame(results_dict)

# # Convert to a GeoDataFrame
# gdf_results = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

# # Output the resulting GeoDataFrame
# gdf_results.head()


In [5]:
# gdf_copy = gdf_results.copy()
# gdf_copy["embeddings_2018"] = [embedding.flatten() if embedding is not None and embedding.size > 0 else None for embedding in gdf_results["embeddings_2018"]]
# gdf_copy["embeddings_2019"] = [embedding.flatten() if embedding is not None and embedding.size > 0 else None for embedding in gdf_results["embeddings_2019"]]

# gdf_copy.to_parquet("train_data/challenge_5.parquet")

In [6]:
gdf = gpd.read_parquet("train_data/challenge_5.parquet")

#### Join flood data

In [7]:
# import geopandas as gpd
# import rasterio
# import xarray as xr
# import numpy as np
# import rioxarray
# from tqdm import tqdm
# from joblib import Parallel, delayed

# # Load your GeoDataFrame
# # gdf = gpd.read_file('your_geopandas_dataframe.gpkg')

# # Load the TIFF files as xarray Datasets
# raster1_path = 'train_data/flood_WM_S1A_IW_GRDH_1SDV_20190816T000150_20190816T000215_028587_033BC6_D6AA.tif'
# raster2_path = 'train_data/flood_WM_S1A_IW_GRDH_1SDV_20190816T000215_20190816T000240_028587_033BC6_4F87.tif'

# # Convert the TIFFs to xarray datasets
# raster1_xr = rioxarray.open_rasterio(raster1_path)
# raster2_xr = rioxarray.open_rasterio(raster2_path)

# def get_raster_value_at_geometry_xr(geometry, raster_xr):
#     try:
#         clipped = raster_xr.rio.clip([geometry], raster_xr.rio.crs)
#         return float(clipped.mean().values)
#     except rioxarray.exceptions.NoDataInBounds:
#         return np.nan

# def process_row(geom, raster_xr):
#     return get_raster_value_at_geometry_xr(geom, raster_xr)

# # Parallel processing with joblib
# n_jobs = 12 # Use all available cores

# # Use tqdm with joblib for progress bar
# raster_values_1 = Parallel(n_jobs=n_jobs)(
#     delayed(process_row)(geom, raster1_xr) for geom in tqdm(gdf['geometry'], desc="Processing Raster 1"))

# raster_values_2 = Parallel(n_jobs=n_jobs)(
#     delayed(process_row)(geom, raster2_xr) for geom in tqdm(gdf['geometry'], desc="Processing Raster 2"))

# # Assign the values back to the GeoDataFrame
# gdf['raster_value_1'] = raster_values_1
# gdf['raster_value_2'] = raster_values_2

# # Set values greater than 1 to NaN for raster_value_1
# gdf.loc[gdf['raster_value_1'] > 1, 'raster_value_1'] = np.nan

# # Set values greater than 1 to NaN for raster_value_2
# gdf.loc[gdf['raster_value_2'] > 1, 'raster_value_2'] = np.nan

# gdf['flood'] = gdf[['raster_value_1', 'raster_value_2']].max(axis=1)

# gdf.to_parquet("train_data/challenge_5.parquet")

Processing Raster 2: 100%|██████████| 6764/6764 [43:06<00:00,  2.62it/s]


In [18]:
gdf['embeddings_delta'] = gdf['embeddings_2019'] - gdf['embeddings_2018']

In [23]:
import pandas as pd
import numpy as np
import optuna
import tensorflow as tf
import joblib
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam

class NeuralNetwork:
    def __init__(self, input_shape, layers, dropout_rate, learning_rate, device):
        self.input_shape = input_shape
        self.layers = layers
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.device = device
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Input(shape=(self.input_shape,)))
        for layer_size in self.layers:
            model.add(Dense(layer_size, activation='relu'))
            model.add(Dropout(self.dropout_rate))
        model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
        return model

    def compile_model(self):
        optimizer = Adam(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Use binary crossentropy for binary classification

    def train_model(self, X_train, y_train, epochs=20, batch_size=32, validation_split=0.2):
        with tf.device(self.device):
            history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        return history

    def evaluate_model(self, X_test, y_test):
        with tf.device(self.device):
            loss, accuracy = self.model.evaluate(X_test, y_test)
        return loss, accuracy

    def predict(self, X_test):
        with tf.device(self.device):
            predictions = self.model.predict(X_test)
        return (predictions > 0.5).astype(int).flatten()  # Convert probabilities to binary predictions

    def calculate_accuracy(self, y_test, predictions):
        return accuracy_score(y_test, predictions)
    
    def save_model(self, filename):
        self.model.save(filename)

    @classmethod
    def load_model(cls, filename, input_shape, device):
        loaded_model = tf.keras.models.load_model(filename)
        nn = cls(input_shape, [], 0, 0, device)  # Dummy values for layers, dropout_rate, and learning_rate
        nn.model = loaded_model
        return nn

def objective(trial):
    layers = []
    for i in range(trial.suggest_int('n_layers', 1, 3)):
        layers.append(trial.suggest_int(f'n_units_l{i}', 64, 512))
    
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    
    nn = NeuralNetwork(input_shape=X_train.shape[1], device=device, layers=layers, dropout_rate=dropout_rate, learning_rate=learning_rate)
    nn.compile_model()
    
    nn.train_model(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
    
    predictions = nn.predict(X_test)
    accuracy = nn.calculate_accuracy(y_test, predictions)
    
    return -accuracy  # We want to maximize accuracy, so minimize negative accuracy

# Assuming combined_gdf is already loaded
combined_gdf_filtered = gdf[['geometry','embeddings_delta','flood']].dropna()

# Flatten the nested arrays
def flatten_embeddings(embedding):
    if isinstance(embedding, np.ndarray) and embedding.ndim > 0:
        return np.concatenate(embedding) if embedding.ndim > 1 else embedding.flatten()
    else:
        return np.array([])  # Handle cases where embedding might be zero-dimensional or empty

X = np.array([flatten_embeddings(x) for x in combined_gdf_filtered['embeddings_delta'] if flatten_embeddings(x).size > 0])
y = combined_gdf_filtered['flood']

# Ensure that X and y have matching lengths after filtering
X = np.array([x for i, x in enumerate(X) if flatten_embeddings(combined_gdf_filtered['embeddings_delta'].iloc[i]).size > 0])
y = y.iloc[:X.shape[0]]  # Adjust y to have the same number of samples as X

# Convert y to binary labels if it's not already
y = (y > 0).astype(int)  # Assuming 'flood' is a numeric value, convert to binary (e.g., flood/no flood)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

# Optimize the hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

# Print the best hyperparameters
print(study.best_params)

# Example usage with the best hyperparameters
best_params = study.best_params
layers = [best_params[f'n_units_l{i}'] for i in range(best_params['n_layers'])]
dropout_rate = best_params['dropout_rate']
learning_rate = best_params['learning_rate']

nn = NeuralNetwork(input_shape=X_train.shape[1], device=device, layers=layers, dropout_rate=dropout_rate, learning_rate=learning_rate)
nn.compile_model()
history = nn.train_model(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
loss, accuracy = nn.evaluate_model(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')
predictions = nn.predict(X_test)
accuracy = nn.calculate_accuracy(y_test, predictions)
print(f'Test Accuracy: {accuracy:.4f}')

# Save the model
nn.save_model('models/task_5_model.h5')

# Save scaler
joblib.dump(scaler, 'models/5_scaler.joblib')


I0000 00:00:1723645330.195529    9023 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
[I 2024-08-14 14:22:10,703] A new study created in memory with name: no-name-ec57d292-8111-4341-aa20-dc94a0650e65
I0000 00:00:1723645330.695229    9023 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723645330.699063    9023 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-

Epoch 1/20


I0000 00:00:1723645330.720866    9023 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723645330.724373    9023 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723645330.903919    9023 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723645330.905425    9023 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

47/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7521 - loss: 0.5496 

I0000 00:00:1723645334.730835   10259 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 1/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.7688 - loss: 0.9398 - val_accuracy: 0.8634 - val_loss: 0.3430
Epoch 2/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8529 - loss: 0.3784 - val_accuracy: 0.8535 - val_loss: 0.2938
Epoch 3/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8629 - loss: 0.3448 - val_accuracy: 0.8720 - val_loss: 0.2749
Epoch 4/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8649 - loss: 0.3373 - val_accuracy: 0.8933 - val_loss: 0.2861
Epoch 5/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8693 - loss: 0.3174 - val_accuracy: 0.8606 - val_loss: 0.3152
Epoch 6/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8640 - loss: 0.3113 - val_accuracy: 0.8691 - val_loss: 0.3521
Epoch 7/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8682 - loss: 0.3174 - val_accuracy: 0.8805 - val_loss: 0.2975
Epoch 8/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8690 - loss: 0.3363 - val_accuracy: 0.8649 - val_loss

[I 2024-08-14 14:23:07,177] Trial 4 finished with value: -0.8361774744027304 and parameters: {'n_layers': 2, 'n_units_l0': 106, 'n_units_l1': 183, 'dropout_rate': 0.4187088874001139, 'learning_rate': 0.00801713255722954}. Best is trial 2 with value: -0.8794084186575654.


{'n_layers': 2, 'n_units_l0': 291, 'n_units_l1': 386, 'dropout_rate': 0.43607258917966674, 'learning_rate': 0.0038200628719317737}
Epoch 1/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.7953 - loss: 0.8618 - val_accuracy: 0.8634 - val_loss: 0.3368
Epoch 2/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8488 - loss: 0.3867 - val_accuracy: 0.8606 - val_loss: 0.2868
Epoch 3/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8689 - loss: 0.3397 - val_accuracy: 0.8691 - val_loss: 0.2800
Epoch 4/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8844 - loss: 0.2639 - val_accuracy: 0.8706 - val_loss: 0.2786
Epoch 5/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8826 - loss: 0.2808 - val_accuracy: 0.8791 - val_loss: 0.2478
Epoch 6/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8865 - loss: 0.2505 - val_accuracy: 0.8862 - val_loss: 0.2742
Epoch 7/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8904 - loss: 0.2681 - val_accuracy: 0.8933 - val

Test Accuracy: 0.8453


['models/5_scaler.joblib']

In [5]:
import geopandas as gpd

# Load the GeoJSON file
geojson_path = 'test_data/challenge_5_bb.geojson'
gdf = gpd.read_file(geojson_path)
gdf

,geometry
0,"POLYGON ((-104.41591 32.61522, -104.41591 32.1..."


In [6]:
import pyproj

def get_utm_zone(longitude):
    return int((longitude + 180) / 6) + 1

# Get the bounds of the geometry
minx, miny, maxx, maxy = gdf.geometry.bounds.iloc[0]

# Calculate UTM zone
utm_zone = get_utm_zone(minx)

# Check for a suitable projection using pyproj
proj = pyproj.Proj(proj='utm', zone=utm_zone, ellps='WGS84')

# Get the corresponding EPSG code for the UTM zone using pyproj
utm_crs = pyproj.CRS(f"+proj=utm +zone={utm_zone} +datum=WGS84")
epsg_code = utm_crs.to_epsg()

# Reproject the GeoDataFrame to the chosen EPSG code
gdf = gdf.to_crs(epsg=epsg_code)
gdf

,geometry
0,"POLYGON ((554799.704 3608782.260, 555059.104 3..."


In [7]:
import numpy as np

# Create a grid of points 5120m apart
x = np.arange(gdf.total_bounds[0], gdf.total_bounds[2], 2560)
y = np.arange(gdf.total_bounds[1], gdf.total_bounds[3], 2560)
xx, yy = np.meshgrid(x, y)
points = np.vstack([xx.ravel(), yy.ravel()]).T

grid = gpd.GeoDataFrame(geometry=gpd.points_from_xy(points[:, 0], points[:, 1], crs=gdf.crs))
grid

,geometry
0,POINT (554799.704 3561297.142)
1,POINT (557359.704 3561297.142)
2,POINT (559919.704 3561297.142)
3,POINT (562479.704 3561297.142)
4,POINT (565039.704 3561297.142)
...,...
394,POINT (595759.704 3607377.142)
395,POINT (598319.704 3607377.142)
396,POINT (600879.704 3607377.142)
397,POINT (603439.704 3607377.142)


In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import pystac_client
import stackstac
import torch
from torchvision import transforms as v2
from box import Box
import yaml
import math
from rasterio.enums import Resampling
from tqdm import tqdm
import rasterio
import warnings
import os
import numpy as np
import rioxarray  # Make sure to import rioxarray to extend xarray

from src.model import ClayMAEModule

warnings.filterwarnings("ignore")

STAC_API = "https://earth-search.aws.element84.com/v1"
COLLECTION = "sentinel-2-l2a"

# Load the model and metadata
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ckpt = "https://clay-model-ckpt.s3.amazonaws.com/v0.5.7/mae_v0.5.7_epoch-13_val-loss-0.3098.ckpt"
torch.set_default_device(device)

torch.cuda.empty_cache()  # Clear GPU cache

# Assuming grid is a GeoDataFrame with the points
points = grid.to_crs("EPSG:4326").geometry.apply(lambda x: (x.x, x.y)).tolist()

model = ClayMAEModule.load_from_checkpoint(
    ckpt, metadata_path="configs/metadata.yaml", shuffle=False, mask_ratio=0
)
model.eval()
model = model.to(device)

metadata = Box(yaml.safe_load(open("configs/metadata.yaml")))

# Function to normalize timestamp
def normalize_timestamp(date):
    week = date.isocalendar().week * 2 * np.pi / 52
    hour = date.hour * 2 * np.pi / 24
    return (math.sin(week), math.cos(week)), (math.sin(hour), math.cos(hour))

# Function to normalize lat/lon
def normalize_latlon(lat, lon):
    lat = lat * np.pi / 180
    lon = lon * np.pi / 180
    return (math.sin(lat), math.cos(lat)), (math.sin(lon), math.cos(lon))

def to_device(data, device):
    if isinstance(data, torch.Tensor):
        return data.to(device)
    elif isinstance(data, dict):
        return {k: to_device(v, device) for k, v in data.items()}
    elif isinstance(data, list):
        return [to_device(v, device) for v in data]
    return data

def process_point(lon, lat, model, metadata, year, device, j):
    model.to(device)  # Ensure the model is on the correct device
    catalog = pystac_client.Client.open(STAC_API)
    search = catalog.search(
        collections=[COLLECTION],
        datetime=f"{year}-06-04/{year}-07-01",
        bbox=(lon - 1e-5, lat - 1e-5, lon + 1e-5, lat + 1e-5),
        max_items=10,
        query={"eo:cloud_cover": {"lt": 80}},
    )

    all_items = search.get_all_items()
    items = list(all_items)
    if not items:
        return None
    
    items = sorted(items, key=lambda x: x.properties.get('eo:cloud_cover', float('inf')))
    lowest_cloud_item = items[0]

    epsg = lowest_cloud_item.properties["proj:epsg"]

    poidf = gpd.GeoDataFrame(
        pd.DataFrame(),
        crs="EPSG:4326",
        geometry=[Point(lon, lat)],
    ).to_crs(epsg)

    coords = poidf.iloc[0].geometry.coords[0]

    size = 256
    gsd = 10
    bounds = (
        coords[0] - (size * gsd) // 2,
        coords[1] - (size * gsd) // 2,
        coords[0] + (size * gsd) // 2,
        coords[1] + (size * gsd) // 2,
    )

    stack = stackstac.stack(
        lowest_cloud_item,
        bounds=bounds,
        snap_bounds=False,
        epsg=epsg,
        resolution=gsd,
        dtype="float32",
        rescale=False,
        fill_value=0,
        assets=["blue", "green", "red", "nir"],
        resampling=Resampling.nearest,
    )

    stack = stack.compute()

    items = []
    dates = []
    for item in all_items:
        if item.datetime.date() not in dates:
            items.append(item)
            dates.append(item.datetime.date())

    platform = "sentinel-2-l2a"
    mean = []
    std = []
    waves = []
    for band in stack.band:
        mean.append(metadata[platform].bands.mean[str(band.values)])
        std.append(metadata[platform].bands.std[str(band.values)])
        waves.append(metadata[platform].bands.wavelength[str(band.values)])

    transform = v2.Compose([v2.Normalize(mean=mean, std=std)])

    datetimes = stack.time.values.astype("datetime64[s]").tolist()
    times = [normalize_timestamp(dat) for dat in datetimes]
    week_norm = [dat[0] for dat in times]
    hour_norm = [dat[1] for dat in times]

    latlons = [normalize_latlon(lat, lon)] * len(times)
    lat_norm = [dat[0] for dat in latlons]
    lon_norm = [dat[1] for dat in latlons]

    pixels = torch.from_numpy(stack.data.astype(np.float32)).to(device)
    pixels = transform(pixels)

    batch_size = 16
    num_batches = math.ceil(len(stack) / batch_size)
    
    embeddings_list = []
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(stack))
        
        batch_pixels = pixels[start_idx:end_idx].to(device)
        batch_time = torch.tensor(np.hstack((week_norm, hour_norm))[start_idx:end_idx], dtype=torch.float32).to(device)
        batch_latlon = torch.tensor(np.hstack((lat_norm, lon_norm))[start_idx:end_idx], dtype=torch.float32).to(device)
        
        batch_datacube = {
            "platform": platform,
            "time": batch_time,
            "latlon": batch_latlon,
            "pixels": batch_pixels,
            "gsd": torch.tensor(stack.gsd.values).to(device),
            "waves": torch.tensor(waves).to(device),
        }

        batch_datacube = to_device(batch_datacube, device)

        try:
            model = model.to(device)

            with torch.no_grad():
                unmsk_patch, _, _, _ = model.model.encoder(batch_datacube)
            batch_embeddings = unmsk_patch[:, 0, :].cpu().numpy()
            embeddings_list.append(batch_embeddings)
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"GPU OOM for point ({lon}, {lat}), batch {i+1}/{num_batches}. Trying CPU...")
                device = torch.device("cpu")
                batch_datacube = to_device(batch_datacube, device)
                model = model.to(device)
                with torch.no_grad():
                    unmsk_patch, _, _, _ = model.model.encoder(batch_datacube)
                batch_embeddings = unmsk_patch[:, 0, :].numpy()
                embeddings_list.append(batch_embeddings)
                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            else:
                raise e

    embeddings = np.concatenate(embeddings_list, axis=0)
    return embeddings

# Initialize an empty dictionary to store results for both years
results_dict = {"lon": [], "lat": [], "embeddings_2017": [], "embeddings_2018": []}

# Specify the years for the datetime range in the search
years = [2017, 2018]

# Iterate through the points and process each one for both years
for i, point in enumerate(tqdm(points)):
    lon, lat = point
    results_dict["lon"].append(lon)
    results_dict["lat"].append(lat)
    
    for year in years:
        embeddings = process_point(lon, lat, model, metadata, year, device, i)
        if embeddings is not None:
            results_dict[f"embeddings_{year}"].append(embeddings)
        else:
            results_dict[f"embeddings_{year}"].append(None)

# Create a DataFrame from the results
df = pd.DataFrame(results_dict)

# Convert to a GeoDataFrame
gdf_results = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

# Output the resulting GeoDataFrame
gdf_results.head()


100%|██████████| 399/399 [11:14<00:00,  1.69s/it]


,lon,lat,embeddings_2017,embeddings_2018,geometry
0,-104.418657,32.186869,"[[0.024514427, -0.019557817, -0.015843777, 0.1...","[[0.0042556943, -0.029356357, -0.045840662, 0....",POINT (-104.41866 32.18687)
1,-104.391500,32.186742,"[[0.022209732, -0.037956085, 0.018563386, 0.18...","[[0.007970055, -0.03960701, -0.0102273, 0.1606...",POINT (-104.39150 32.18674)
2,-104.364344,32.186608,"[[0.06547231, -0.0403255, 0.03680599, 0.102555...","[[0.05164792, -0.038914327, 0.033683784, 0.115...",POINT (-104.36434 32.18661)
3,-104.337188,32.186469,"[[-0.042644504, 0.0076282974, 0.07318993, 0.11...","[[-0.040943988, 0.023629641, 0.06421219, 0.124...",POINT (-104.33719 32.18647)
4,-104.310032,32.186323,"[[-0.014203708, 0.038455915, 0.07248832, 0.094...","[[-0.003047574, 0.045992475, 0.06563147, 0.110...",POINT (-104.31003 32.18632)


In [9]:
gdf_copy = gdf_results.copy()
gdf_copy["embeddings_2017"] = [embedding.flatten() if embedding is not None and embedding.size > 0 else None for embedding in gdf_results["embeddings_2017"]]
gdf_copy["embeddings_2018"] = [embedding.flatten() if embedding is not None and embedding.size > 0 else None for embedding in gdf_results["embeddings_2018"]]

gdf_copy.to_parquet("test_data/challenge_5.parquet")

In [10]:
import geopandas as gpd
import rasterio
import xarray as xr
import numpy as np
import rioxarray
from tqdm import tqdm
from joblib import Parallel, delayed

# Load your GeoDataFrame
# gdf = gpd.read_file('your_geopandas_dataframe.gpkg')

# Load the TIFF files as xarray Datasets
raster_path = 'test_data/flood_WM_S1A_IW_GRDH_1SDV_20180604T125505_20180604T125530_022207_026702_642A.tif'

# Convert the TIFFs to xarray datasets
raster_xr = rioxarray.open_rasterio(raster_path)

def get_raster_value_at_geometry_xr(geometry, raster_xr):
    try:
        clipped = raster_xr.rio.clip([geometry], raster_xr.rio.crs)
        return float(clipped.mean().values)
    except rioxarray.exceptions.NoDataInBounds:
        return np.nan

def process_row(geom, raster_xr):
    return get_raster_value_at_geometry_xr(geom, raster_xr)

# Parallel processing with joblib
n_jobs = 12 # Use all available cores

# Use tqdm with joblib for progress bar
raster_values = Parallel(n_jobs=n_jobs)(
    delayed(process_row)(geom, raster_xr) for geom in tqdm(gdf_copy['geometry'], desc="Processing Raster 1"))

# Assign the values back to the GeoDataFrame
gdf_copy['flood'] = raster_values

# Set values greater than 1 to NaN for raster_value_1
gdf_copy.loc[gdf_copy['flood'] > 1, 'flood'] = np.nan

gdf_copy['embeddings_delta'] = gdf_copy['embeddings_2018'] - gdf_copy['embeddings_2017']

gdf_copy.to_parquet("test_data/challenge_5.parquet")

Processing Raster 1: 100%|██████████| 399/399 [02:39<00:00,  2.51it/s]


KeyError: 'flood'

In [14]:
import pandas as pd
import numpy as np
import optuna
import tensorflow as tf
import joblib
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam

class NeuralNetwork:
    def __init__(self, input_shape, layers, dropout_rate, learning_rate, device):
        self.input_shape = input_shape
        self.layers = layers
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.device = device
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Input(shape=(self.input_shape,)))
        for layer_size in self.layers:
            model.add(Dense(layer_size, activation='relu'))
            model.add(Dropout(self.dropout_rate))
        model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
        return model

    def compile_model(self):
        optimizer = Adam(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Use binary crossentropy for binary classification

    def train_model(self, X_train, y_train, epochs=20, batch_size=32, validation_split=0.2):
        with tf.device(self.device):
            history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        return history

    def evaluate_model(self, X_test, y_test):
        with tf.device(self.device):
            loss, accuracy = self.model.evaluate(X_test, y_test)
        return loss, accuracy

    def predict(self, X_test):
        with tf.device(self.device):
            predictions = self.model.predict(X_test)
        return (predictions > 0.5).astype(int).flatten()  # Convert probabilities to binary predictions

    def calculate_accuracy(self, y_test, predictions):
        return accuracy_score(y_test, predictions)
    
    def save_model(self, filename):
        self.model.save(filename)

    @classmethod
    def load_model(cls, filename, input_shape, device):
        loaded_model = tf.keras.models.load_model(filename)
        nn = cls(input_shape, [], 0, 0, device)  # Dummy values for layers, dropout_rate, and learning_rate
        nn.model = loaded_model
        return nn

def objective(trial):
    layers = []
    for i in range(trial.suggest_int('n_layers', 1, 3)):
        layers.append(trial.suggest_int(f'n_units_l{i}', 64, 512))
    
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    
    nn = NeuralNetwork(input_shape=X_train.shape[1], device=device, layers=layers, dropout_rate=dropout_rate, learning_rate=learning_rate)
    nn.compile_model()
    
    nn.train_model(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
    
    predictions = nn.predict(X_test)
    accuracy = nn.calculate_accuracy(y_test, predictions)
    
    return -accuracy  # We want to maximize accuracy, so minimize negative accuracy

In [16]:
# Detect if GPU is available
device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

# Load the model
loaded_nn = NeuralNetwork.load_model('models/task_5_model.h5', input_shape=768, device=device)

# Prepare your new data (assuming it's in the same format as your training data)
new_data = np.squeeze(gdf_copy['embeddings_delta'].tolist())
new_data = pd.DataFrame(new_data)  # Ensure the new data is in DataFrame format

# Make predictions
new_predictions = loaded_nn.predict(new_data)

gdf_copy['pred_flood'] = new_predictions

I0000 00:00:1723648156.989605   30624 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723648156.991492   30624 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723648156.992914   30624 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723648157.000614   30624 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

 1/13 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step

I0000 00:00:1723648159.281550   31258 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step


In [17]:
print("Test set accuracy:", np.mean(gdf_copy['pred_flood']==gdf_copy['flood']))

Test set accuracy: 0.6290726817042607
